In [1]:
import string
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.mode.chained_assignment = None

In [2]:
df=pd.read_csv('Extracted_data.csv')
display(df.shape)
df.head()

(5991, 8)

,Author,Book_Name,Book_Category,Book_url,Publishing_house,Book_size,Book_pages,Quotation
0,الكاتب محمود سالم,كتاب لغز القبر الملكي PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2 ميغابايت,52 صفحة,NaN
1,الكاتب محمود سالم,كتاب لغز الكوخ المحترق PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2 ميغابايت,56 صفحة,NaN
2,الكاتب محمود سالم,كتاب لغز اللص الشبح PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2 ميغابايت,48 صفحة,NaN
3,الكاتب محمود سالم,كتاب لغز المدينة العائمة PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2 ميغابايت,54 صفحة,NaN
4,الكاتب محمود سالم,كتاب لغز المنزل رقم 98 PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2 ميغابايت,48 صفحة,NaN


In [3]:
df.isna().sum()

Author               251
Book_Name              4
Book_Category         59
Book_url               4
Publishing_house     702
Book_size              5
Book_pages             5
Quotation           5801
dtype: int64

In [4]:
quotation_null=df["Quotation"].isna().sum() / df.shape[0] *100
print(f"Quotation Column is {quotation_null}% empty")
# we will ignore it for now

Quotation Column is 96.82857619762977% empty


In [5]:
#check other null data
df[df["Book_pages"].isna() | df["Book_size"].isna() | df["Book_url"].isna()]

,Author,Book_Name,Book_Category,Book_url,Publishing_house,Book_size,Book_pages,Quotation
3314,الكاتب ابراهيم السكران,كتاب التعبد لله بتقبيل أقدام الولاة PDF,NaN,https://www.arab-books.com/books/%d9%83%d8%aa%...,NaN,NaN,NaN,NaN
3967,كتب الامام الالباني,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4024,الكاتب توفيق الحكيم,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df=df[~(df["Book_pages"].isna() | df["Book_size"].isna() | df["Book_url"].isna())]
df.isna().sum()

Author               249
Book_Name              0
Book_Category         54
Book_url               0
Publishing_house     697
Book_size              0
Book_pages             0
Quotation           5796
dtype: int64

In [7]:
# we will use the books with missing category later to test our model, so we will leave it for now and start cleaning text data
df

,Author,Book_Name,Book_Category,Book_url,Publishing_house,Book_size,Book_pages,Quotation
0,الكاتب محمود سالم,كتاب لغز القبر الملكي PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2 ميغابايت,52 صفحة,NaN
1,الكاتب محمود سالم,كتاب لغز الكوخ المحترق PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2 ميغابايت,56 صفحة,NaN
2,الكاتب محمود سالم,كتاب لغز اللص الشبح PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2 ميغابايت,48 صفحة,NaN
3,الكاتب محمود سالم,كتاب لغز المدينة العائمة PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2 ميغابايت,54 صفحة,NaN
4,الكاتب محمود سالم,كتاب لغز المنزل رقم 98 PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2 ميغابايت,48 صفحة,NaN
...,...,...,...,...,...,...,...,...
5986,الكاتب فرانز كافكا,رواية التحول فرانتس كافكا pdf,كتب الروايات العالمية المترجمة,https://www.arab-books.com/books/%d8%b1%d9%88%...,منشورات الجمل,1.64 ميغابايت,106 صفحة,NaN
5987,الشيخ عبدالوهاب خلاف,كتاب احكام الاحوال الشخصية في الشريعة الاسلامية,كتب أصول الفقه,https://www.arab-books.com/books/%d9%83%d8%aa%...,دار الكتب العلمية,14 ميغابايت,289 صفحة,NaN
5988,الله عز وجل,Corán traducido al español PDF – القران نسخة ا...,القرآن الكريم,https://www.arab-books.com/books/coran-traduci...,محمد رسول الله,26 ميغابايت,1088 صفحة,NaN
5989,الله عز وجل,القران الكريم باللغة الانجليزية -Quran with En...,القرآن الكريم,https://www.arab-books.com/books/%d8%a7%d9%84%...,محمد رسول الله,30 ميغابايت,978 صفحة,NaN


In [8]:
df["Book_size"]=df["Book_size"].str.replace(r'[^\d.]','',regex=True)
df.rename(columns={'Book_size':'Book_size_MB'},inplace=True)
df.head()

,Author,Book_Name,Book_Category,Book_url,Publishing_house,Book_size_MB,Book_pages,Quotation
0,الكاتب محمود سالم,كتاب لغز القبر الملكي PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,52 صفحة,NaN
1,الكاتب محمود سالم,كتاب لغز الكوخ المحترق PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,56 صفحة,NaN
2,الكاتب محمود سالم,كتاب لغز اللص الشبح PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,48 صفحة,NaN
3,الكاتب محمود سالم,كتاب لغز المدينة العائمة PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,54 صفحة,NaN
4,الكاتب محمود سالم,كتاب لغز المنزل رقم 98 PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,48 صفحة,NaN


In [9]:
df["Book_pages"]=df["Book_pages"].str.replace(r'[^\d]','',regex=True)
df.head()

,Author,Book_Name,Book_Category,Book_url,Publishing_house,Book_size_MB,Book_pages,Quotation
0,الكاتب محمود سالم,كتاب لغز القبر الملكي PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,52,NaN
1,الكاتب محمود سالم,كتاب لغز الكوخ المحترق PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,56,NaN
2,الكاتب محمود سالم,كتاب لغز اللص الشبح PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,48,NaN
3,الكاتب محمود سالم,كتاب لغز المدينة العائمة PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,54,NaN
4,الكاتب محمود سالم,كتاب لغز المنزل رقم 98 PDF,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,48,NaN


In [10]:
book_name=[]
for word in df['Book_Name']:
    phase1=re.sub(r'[Pp][Dd][Ff]|كتاب|[!":,.؟]','',word)
    phase2=phase1.split('للكاتب')[0]
    phase3=re.sub(r'\s{2,}',' ',phase2)
    phase4=phase3.strip()
    book_name.append(phase4)

df['Book_Name']=book_name
df=df[~df['Book_Name'].str.match(u'[^\u0621-\u064A\u0660-\u0669a-zA-Z0-9]')]
df

,Author,Book_Name,Book_Category,Book_url,Publishing_house,Book_size_MB,Book_pages,Quotation
0,الكاتب محمود سالم,لغز القبر الملكي,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,52,NaN
1,الكاتب محمود سالم,لغز الكوخ المحترق,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,56,NaN
2,الكاتب محمود سالم,لغز اللص الشبح,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,48,NaN
3,الكاتب محمود سالم,لغز المدينة العائمة,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,54,NaN
4,الكاتب محمود سالم,لغز المنزل رقم 98,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,48,NaN
...,...,...,...,...,...,...,...,...
5986,الكاتب فرانز كافكا,رواية التحول فرانتس كافكا,كتب الروايات العالمية المترجمة,https://www.arab-books.com/books/%d8%b1%d9%88%...,منشورات الجمل,1.64,106,NaN
5987,الشيخ عبدالوهاب خلاف,احكام الاحوال الشخصية في الشريعة الاسلامية,كتب أصول الفقه,https://www.arab-books.com/books/%d9%83%d8%aa%...,دار الكتب العلمية,14,289,NaN
5988,الله عز وجل,Corán traducido al español – القران نسخة اسبانية,القرآن الكريم,https://www.arab-books.com/books/coran-traduci...,محمد رسول الله,26,1088,NaN
5989,الله عز وجل,القران الكريم باللغة الانجليزية -Quran with En...,القرآن الكريم,https://www.arab-books.com/books/%d8%a7%d9%84%...,محمد رسول الله,30,978,NaN


In [11]:
df["Publishing_house"]=df["Publishing_house"].str.replace(u'[^\u0621-\u064A\u0660-\u0669\s]','',regex=True)
df

,Author,Book_Name,Book_Category,Book_url,Publishing_house,Book_size_MB,Book_pages,Quotation
0,الكاتب محمود سالم,لغز القبر الملكي,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,52,NaN
1,الكاتب محمود سالم,لغز الكوخ المحترق,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,56,NaN
2,الكاتب محمود سالم,لغز اللص الشبح,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,48,NaN
3,الكاتب محمود سالم,لغز المدينة العائمة,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,54,NaN
4,الكاتب محمود سالم,لغز المنزل رقم 98,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,48,NaN
...,...,...,...,...,...,...,...,...
5986,الكاتب فرانز كافكا,رواية التحول فرانتس كافكا,كتب الروايات العالمية المترجمة,https://www.arab-books.com/books/%d8%b1%d9%88%...,منشورات الجمل,1.64,106,NaN
5987,الشيخ عبدالوهاب خلاف,احكام الاحوال الشخصية في الشريعة الاسلامية,كتب أصول الفقه,https://www.arab-books.com/books/%d9%83%d8%aa%...,دار الكتب العلمية,14,289,NaN
5988,الله عز وجل,Corán traducido al español – القران نسخة اسبانية,القرآن الكريم,https://www.arab-books.com/books/coran-traduci...,محمد رسول الله,26,1088,NaN
5989,الله عز وجل,القران الكريم باللغة الانجليزية -Quran with En...,القرآن الكريم,https://www.arab-books.com/books/%d8%a7%d9%84%...,محمد رسول الله,30,978,NaN


In [12]:
df.replace(r'^\s*$', np.NaN, regex=True,inplace=True)

In [13]:
df.head(20)

,Author,Book_Name,Book_Category,Book_url,Publishing_house,Book_size_MB,Book_pages,Quotation
0,الكاتب محمود سالم,لغز القبر الملكي,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,52,NaN
1,الكاتب محمود سالم,لغز الكوخ المحترق,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,56,NaN
2,الكاتب محمود سالم,لغز اللص الشبح,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,48,NaN
3,الكاتب محمود سالم,لغز المدينة العائمة,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,54,NaN
4,الكاتب محمود سالم,لغز المنزل رقم 98,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,48,NaN
5,الكاتب محمود سالم,لغز الموسيقار الصغير,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,54,NaN
6,الكاتب محمود سالم,لغز الوثائق السرية,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2,50,NaN
7,الكاتب هارون يحيى,التضحية عند الحيوان,كتب الأدب العربي-كتب متنوعة,https://www.arab-books.com/books/%d9%83%d8%aa%...,عدنان أوكطار,5,160,ترون بأنفسكم نماذج من السلوك عند بعض الحيوانات...
8,الكاتب هارون يحيى,معجزة الذرة,كتب تطوير الذات-كتب متنوعة,https://www.arab-books.com/books/%d9%83%d8%aa%...,عدنان أوكطار,3.41,136,إن هذا الكتاب يبين وبالدليل العلمي أن الذي خلق...
9,الكاتب هارون يحيى,لا تتجاهل,كتب الأدب العربي,https://www.arab-books.com/books/%d9%83%d8%aa%...,عدنان أوكطار,3,100,هذه الحقيقة بالإضافة إلى حقائق كونية أخرى يستع...


In [14]:
df.to_csv('Cleaned_data.csv',encoding='utf-8-sig',index=False)